In [88]:
import re
import nltk
import numpy as np
import pandas as pd
import pickle as pkl
from collections import defaultdict
from gensim.models import Word2Vec
from functools import reduce
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from annotation import load_df_pickle

In [2]:
tokenizer = RegexpTokenizer(r'\w+')
ps = PorterStemmer()

In [3]:
data = load_df_pickle('./cleandatanew.pkl')
data = data.reset_index().drop('index', axis=1)
data['subsections'] = data['subsections'].apply(lambda x: x.replace(';', ' '))
data = data[['docno', 'title', 'abstract', 'subsections']]

dataframe loaded, columns:  Index(['docno', 'conference', 'year', 'workshop', 'title', 'authors',
       'pdf_link', 'arxiv_link', 'supp_link', 'abstract', 'subsections'],
      dtype='object')


In [4]:
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not","can't": "can not","can't've": "cannot have",
"'cause": "because","could've": "could have","couldn't": "could not","couldn't've": "could not have",
"didn't": "did not","doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have",
"hasn't": "has not","haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will",
"he'll've": "he will have","how'd": "how did","how'd'y": "how do you","how'll": "how will","i'd": "i would",
"i'd've": "i would have","i'll": "i will","i'll've": "i will have","i'm": "i am","i've": "i have",
"isn't": "is not","it'd": "it would","it'd've": "it would have","it'll": "it will","it'll've": "it will have",
"let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not",
"mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have",
"needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
"oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
"shan't've": "shall not have","she'd": "she would","she'd've": "she would have","she'll": "she will",
"she'll've": "she will have","should've": "should have","shouldn't": "should not",
"shouldn't've": "should not have","so've": "so have","that'd": "that would","that'd've": "that would have",
"there'd": "there would","there'd've": "there would have",
"they'd": "they would","they'd've": "they would have","they'll": "they will","they'll've": "they will have",
"they're": "they are","they've": "they have","to've": "to have","wasn't": "was not","we'd": "we would",
"we'd've": "we would have","we'll": "we will","we'll've": "we will have","we're": "we are","we've": "we have",
"weren't": "were not","what'll": "what will","what'll've": "what will have","what're": "what are",
"what've": "what have","when've": "when have","where'd": "where did",
"where've": "where have","who'll": "who will","who'll've": "who will have","who've": "who have",
"why've": "why have","will've": "will have","won't": "will not","won't've": "will not have",
"would've": "would have","wouldn't": "would not","wouldn't've": "would not have","y'all": "you all",
"y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
"you'd": "you would","you'd've": "you would have","you'll": "you will","you'll've": "you will have",
"you're": "you are","you've": "you have"}

contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

# Function for expanding contractions
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

In [5]:
# preprocessing
# clean subsections
data['subsections']

0        acknowledgements architectural details ilsvrc ...
1        introduction related work bilateral filtering ...
2        introduction related work fast clustering on b...
3        introduction related work incremental learning...
4        introduction supervised discrete hashing appro...
                               ...                        
15296    introduction related works zero-shot learning ...
15297    introduction our contributions problem related...
15298    introduction related work proposed method expe...
15299    introduction related work methodology gcn trai...
15300    introduction related works human motion predic...
Name: subsections, Length: 15301, dtype: object

In [6]:
stops = set(stopwords.words('english'))
data['subsections_token'] = data['subsections'].apply(lambda s: [ps.stem(t) for t in tokenizer.tokenize(s) if t not in stops])

In [7]:
df = defaultdict(int)
for i in range(len(data)):
    for t in set(data.loc[i, 'subsections_token']):
        df[t] += 1

In [8]:
sub_stops = set(map(lambda x: x[1], sorted([[v,k] for k, v in df.items()], reverse=True)[:15]))
data['subsections_token'] = data['subsections_token'].apply(lambda l: [t for t in l if t not in sub_stops])

In [9]:
data['subsections_token']

0        [acknowledg, architectur, ilsvrc, 2014, classi...
1        [bilater, filter, guid, filter, geodes, filter...
2        [fast, cluster, binari, hash, code, k, mean, h...
3        [increment, learn, framework, background, larg...
4        [supervis, discret, hash, approxim, bi, nonlin...
                               ...                        
15296    [zero, shot, learn, contrast, learn, propos, a...
15297    [contribut, problem, convolut, neural, network...
15298    [propos, sampl, graph, construct, graph, lapla...
15299    [methodolog, gcn, gcn, predict, methodolog, gc...
15300    [human, motion, predict, graph, represent, lea...
Name: subsections_token, Length: 15301, dtype: object

In [71]:
# delete stop words
# tokenize
# remove pure numbers
stops = set(stopwords.words('english'))
data['title_token'] = data['title'].apply(lambda s: [ps.stem(t) for t in tokenizer.tokenize(s) if t not in stops])
pure_num = re.compile('\d+')
data['abstract_token'] = data['abstract'].apply(lambda s: [ps.stem(t) for t in tokenizer.tokenize(s) if t not in stops and re.match(pure_num, t) is None])

In [72]:
abstract_sent_token = data['abstract'].apply(nltk.sent_tokenize).apply(pd.Series).unstack().reset_index(drop=True).dropna()

In [73]:
abstract_sent_token = abstract_sent_token.apply(lambda s: [ps.stem(t) for t in tokenizer.tokenize(s) if t not in stops and re.match(pure_num, t) is None])

In [74]:
abstract_sent_token

0         [we, propos, deep, convolut, neural, network, ...
1         [in, paper, propos, propag, filter, novel, ima...
2         [thi, paper, address, problem, cluster, larg, ...
3         [over, last, sever, year, shown, imag, base, o...
4         [recent, learn, base, hash, techniqu, attract,...
                                ...                        
243481    [we, examin, small, exampl, use, procedur, ana...
246906                                              [intel]
252381    [second, real, time, semant, segment, exampl, ...
252398    [also, result, better, filter, scheme, vs, vim...
253415    [notabl, achiev, place, public, leaderboard, s...
Length: 115058, dtype: object

In [76]:
corpus = list(abstract_sent_token) + list(data['subsections_token']) + list(data['title_token'])
with open('corpus.pkl', 'wb') as f:
    pkl.dump(corpus, f)

In [78]:
with open('corpus.pkl', 'rb') as f:
    corpus = pkl.load(f)
len(corpus)

145660

In [93]:
model = Word2Vec(sentences=corpus, vector_size=128, window=5, min_count=1, workers=4, epochs=10)

In [94]:
model.wv.save('word2vec.wv')

In [95]:
wv = model.wv

In [107]:
# do sum pooling to the title, abstract and subsections
for f in ['title', 'abstract', 'subsections']:
    data[f + '_embedding'] = data[f + '_token'].apply(lambda l: np.sum([wv[t] for t in l], axis=0))

In [108]:
for f in ['title', 'abstract', 'subsections']:
    data[f + '_embedding'] = data[f + '_embedding'].apply(lambda x: ' '.join(x.astype('str')))

In [112]:
data[['docno', 'title_embedding', 'abstract_embedding', 'subsections_embedding']].to_csv('./word2vec_embedding_df.csv', index=False)